In [2]:
import jdc

In [3]:
import gym
env=gym.make('MsPacman-v4')
state=env.reset()
#env.render()



### MsPacman 

state는 (210,160,3)으로 210x160 픽셀 컬러 이미지이다. 

계산 용의를 위해 픽셀 크기 절반으로 줄이고 흑백이미지를 넣어준다. 

모델 input_shape에 맞게 dimension을 조절해준다.



In [4]:
import numpy as np
print(state.shape)
state_1 = state[1:176:2,::2]
print(state_1.shape)
state_1 = state_1.mean(axis=2)
state_1 = state_1.reshape(88,80,1)
print(state_1.shape)
state_1 = np.expand_dims(state_1,axis=0)
print(state_1.shape)
print(env.action_space)

(210, 160, 3)
(88, 80, 3)
(88, 80, 1)
(1, 88, 80, 1)
Discrete(9)


Q 함수를 딥러닝으로 추정하기 위해 필요한 tensorflow.keras 라이브러리와 
replay buffer를 만들기 위한 deque를 호출하고,

DQN class의 argument를 제공하기 위해 state_shape와 action_size의 값을 준다. 또, process_state()를 통해 위의 셀으 코드를 함수화 한다.

In [54]:
import random
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten
from tensorflow.keras.optimizers import Adam
state_shape = (88,80,1)
action_size = env.action_space.n
color = np.array([210,164,74]).mean

def process_state(state):
    img = state[1:176:2,::2]  # down shape
    img = img.mean(axis=2)    # black image
    img[img==color]=0         #    
    img=(img-128)/128  # pixel 값을 -1~1로 바꿔주고 
    img = img.reshape(88,80,1)
    return img

# DQN class

## 5개의 내부 함수로 구현

replay_buffer 크기는 5000,

$ \gamma $ = 0.9,

target network는 1000 time step에 한번씩 update, 

#### 첫번째 함수 
def __init__() 내에 main_network과 target_network으로 객체화하여 DQN 클래스 속성으로 정의하고, main_network의 초기치를 target_network의 추정치로 copy한다.


DQN 클래스의 첫 번째 함수 q_network()는 크기가 (batch,88,80,1)인 4D 텐서 이미지 자료를 Input으로 받는다.

Hidden layer는 아래 코드와 같이 구성,

Flatten으로 array를 펼쳐주고, DNN으로 연결해주고

Output Data는 (batch,action_size)를 출력한다.

이는 $Q_\theta(s,a)$를 출력하는 것으로, actio_size=9이므로 batch=8로 가정하면 (8,9)인 2D Tensor를 출력한다.

끝으로 손실함수는 목적변수 $r_i+\gamma max_{a'}Q_{\theta '}(s_i',a') $ 와 $Q_{\theta}(s,a)$의 MSE로 정의하고 있다.





In [58]:
class DQN:
    def __init__(self, state_shape,action_size):
        self.state_shape = state_shape
        self.action_size = action_size
        self.replay_buffer = deque(maxlen=5000)
        self.gamma = 0.9
        self.update_timesteps = 1000
        self.eps_min = 0.1
        self.eps_max = 0.8
        self.eps_steps = 2000000
        self.main_network = self.q_network()
        self.target_network = self.q_network()
        self.target_network.set_weights = (self.main_network.get_weights())
        
    def q_network(self):
        state_shape = (88,80,1)
        model = Sequential()
        model.add(Conv2D(32,8,strides=4,padding='same',activation='relu',
                        input_shape=state_shape))
        model.add(Conv2D(64,4,strides=2,padding='same',activation='relu'))
        model.add(Conv2D(64,3,padding='same',activation='relu'))
        model.add(Flatten())
        model.add(Dense(512,activation='relu'))
        model.add(Dense(128,activation='relu'))
        model.add(Dense(self.action_size,activation='linear'))

        model.compile(loss='mse',optimizer=Adam())
        return model

#### 두 번째 함수
store_transition()는 (state,action,reward,next_state,done)을 입력받아 replay_buffer에 저장하는 함수이다. 

#### 세 번째 함수
epsilon_greedy() 함수로 epslion-greedy policy를 정의하고 있다.

이 함수는 time-step에 따라 $\epsilon$을 선형적으로 감소시켜 학습초기에는 임의의 action을 선택하는 비율이 높고 점차적으로 임의로 선택하는 비율을 낮추도록 scheduling 되어 있다.




In [76]:
%%add_to DQN
def store_transitions(self, state, action, reward, next_state, done):
    self.replay_buffer.append((state,action,reward,next_state,done))
    
def epsilon_greedy(self,state,step):
    epsilon = max(self.eps_min, self.eps_max-(self.eps_max-self.eps_min)
                 *step/self.eps_steps)
    if random.uniform(0,1) < epsilon:
        return np.random.randint(self.action_size)
    else:
        state = state[np.newaxis,:]
        Q_values = self.main_network.predict(state)
        return np.argmax(Q_values[0])
 

#### 네 번째 함수
train()은 $Q_{\theta}(s,a)$를 학습시키는 함수로, replay_buffer로부터 batch_size만큼 sample indices를 임의로 추출하여, 
즉,$(s_i,a_i,r_i,s_{i+1},a_{i+1})$에 대응하는 index i를 batch_size만큼 임의로 추출해준다.

target_Q_values를 $r_i+\gamma max_{a'}Q_{\theta '}(s_i',a')$으로 정의하면 입력된 state에서 오직 하나의 action만 target_Q_values에 존재한다. 

예를들어 8개의 state가 입력되면 출력되는 action이 각 state당 9개 이지만 target Q value는 크기가 (8,) 인 1D 텐서로 구성되어 있다. 

각 state에서 오직 하나의 action에만 대응하는 목적변수 값만 있으므로 각 state의 나머지 8개의 action의 값을 지정해야 main_network의 출력 shape과 일치 시킬 수 있게된다. 나머지 action에 대한 목적변수 값을 main_network의 예측값과 같게 지정하면 이들의 MSE는 0이 된다. 

또한 batch 단위로 학습시켜야 하므로 train_on_batch() 함수를 이용해 학습시키고 있다. 

#### 다섯 번째 함수
update_network()으로 target_network의 모수를 main_network의 모수로 복사하는 함수이다. 

In [91]:
%%add_to DQN

def train(self,batch_size):
    indices = np.random.randint(len(self.replay_buffer), size= batch_size)
    batch = [self.replay_buffer[index] for index in indices]
    
    states,actions,reward,next_states,dones = [
                            np.array([experience[field_index] for experience in batch])
                                for field_index in range(5)]
    next_Q_values = self.target_network.predict(next_states)
    max_next_Q_values = np.max(next_Q_values, axis = 1)
    target_Q_values = (reward + (1-dones)*self.gamma*max_next_Q_values)
    
    y_Q_values = self.main_network.predict(states)
    for k, action in enumerate(actions):
        y_Q_values[k][action] = target_Q_values[k]
    
    #states = np.expand_dims(state,axis=0)
    
    self.main_network.train_on_batch(states,y_Q_values)

def update_target_network(self):
    self.target_network.set_weights = (self.main_network.get_weights())

    

## 학습

이제 DQN 학습을 시켜보겠다.

state는 process_state()를 이용해 자료를 사전정리하고 target_network의 weight는 1,000번 time-step마다 main_network의 weight로 update하도록 한다.

총 episode는 500번이지만, 일반적인 컴퓨터로는 많은 시간이 소요된다.


프로그램에서 training_interval = 4로 준 것이 있다.

팩맨 게임에 높은 점수를 얻기 위해서는 ghost가 움직이는 방향을 미리 파악해서 미리 피해야 한다.

정지된 1장의 사진으로는 움직임 파악을 못하므로 매 4번째 간격의 사진들을 state로 받아온다. 고로 ghost의 방향을 파악해서 DQN의 성능 향상에 기여한다.

최초 10개의 episode에서 획득한 점수를 보면 초기 단계의 학습이라 낮은 점수를 획득한다. 


epsilon_greedy() 함수에서 $\epsilon$을 episode가 증가할수록 감소시키고 있기 때문에 갈수록 action을 greedy하게 선택할 확률이 높아진다.

결국, 학습이 잘 되고있다면 return이 증가 할 것이다. 







In [94]:
import time
num_episodes = 500
num_timesteps = 20000
batch_size= 8
training_interval = 4
return_monitor = []
dqn = DQN(state_shape,action_size)

done = False
time_step=0
t_start = time.time()

for i in range(num_episodes):
    Return=0
    state = process_state(env.reset())
    for t in range(num_timesteps):
        env.render()
        time_step += 1
        if time_step % dqn.update_timesteps == 0:
            dqn.update_target_network()
        
        action = dqn.epsilon_greedy(state,t*(i+1))
        next_state,reward,done,_ = env.step(action)
        next_state = process_state(next_state)
        
        dqn.store_transitions(state,action,reward,next_state,done)
        
        state = next_state
        
        Return += reward
        
        if time_step % training_interval != 0:
            continue
        
        if done:
            print('Episode: ',i, ",",'return: ', Return)
            return_monitor.append([i,Return])
            break
        
        if len(dqn.replay_buffer) > batch_size:
            dqn.train(batch_size)
        
    
print(time.time() - t_start)

(88, 80, 1)
Episode:  0 , return:  320.0
1648547687.7133398
(88, 80, 1)
Episode:  1 , return:  210.0
1648547961.5615108
(88, 80, 1)
Episode:  2 , return:  200.0
1648548008.3026273
(88, 80, 1)
Episode:  3 , return:  160.0
1648548000.0920827
(88, 80, 1)
Episode:  4 , return:  210.0
1648548049.8571112
(88, 80, 1)
Episode:  5 , return:  200.0
1648548144.6516113
(88, 80, 1)
Episode:  6 , return:  270.0
1648548106.2390509
(88, 80, 1)
Episode:  7 , return:  170.0
1648548185.7021987
(88, 80, 1)
Episode:  8 , return:  420.0
1648547809.1670547
(88, 80, 1)
Episode:  9 , return:  230.0
1648548134.3304896
(88, 80, 1)
Episode:  10 , return:  650.0
1648547778.1193094
(88, 80, 1)
Episode:  11 , return:  910.0
1648547675.4314396
(88, 80, 1)
Episode:  12 , return:  580.0
1648547989.0606718
(88, 80, 1)
Episode:  13 , return:  360.0
1648548149.5521097
(88, 80, 1)
Episode:  14 , return:  220.0
1648548252.1122382
(88, 80, 1)
Episode:  15 , return:  270.0
1648548338.3761032
(88, 80, 1)
Episode:  16 , return:

Episode:  135 , return:  220.0
1648551242.9178588
(88, 80, 1)
Episode:  136 , return:  290.0
1648551185.6393235
(88, 80, 1)
Episode:  137 , return:  230.0
1648551292.8311176
(88, 80, 1)
Episode:  138 , return:  400.0
1648551235.1684382
(88, 80, 1)
Episode:  139 , return:  300.0
1648551338.4069989
(88, 80, 1)
Episode:  140 , return:  230.0
1648551365.9668353
(88, 80, 1)
Episode:  141 , return:  170.0
1648551447.3469737
(88, 80, 1)
Episode:  142 , return:  300.0
1648551495.711998
(88, 80, 1)
Episode:  143 , return:  300.0
1648551335.1621633
(88, 80, 1)
Episode:  144 , return:  140.0
1648551648.1585186
(88, 80, 1)
Episode:  145 , return:  160.0
1648551614.664772
(88, 80, 1)
Episode:  146 , return:  350.0
1648551425.0535617
(88, 80, 1)
Episode:  147 , return:  230.0
1648551636.4688494
(88, 80, 1)
Episode:  148 , return:  220.0
1648551648.314912
(88, 80, 1)
Episode:  149 , return:  240.0
1648551559.7934823
(88, 80, 1)
Episode:  150 , return:  230.0
1648551679.9659934
(88, 80, 1)
Episode:  1

Episode:  268 , return:  320.0
1648554491.7794206
(88, 80, 1)
Episode:  269 , return:  260.0
1648554408.8254752
(88, 80, 1)
Episode:  270 , return:  270.0
1648554584.2250187
(88, 80, 1)
Episode:  271 , return:  300.0
1648554434.5506523
(88, 80, 1)
Episode:  272 , return:  130.0
1648554622.6525335
(88, 80, 1)
Episode:  273 , return:  170.0
1648554690.057139
(88, 80, 1)
Episode:  274 , return:  310.0
1648554712.1380742
(88, 80, 1)
Episode:  275 , return:  330.0
1648554599.4649582
(88, 80, 1)
Episode:  276 , return:  340.0
1648554696.996555
(88, 80, 1)
Episode:  277 , return:  360.0
1648554595.931248
(88, 80, 1)
Episode:  278 , return:  300.0
1648554714.7319796
(88, 80, 1)
Episode:  279 , return:  170.0
1648554984.1650844
(88, 80, 1)
Episode:  280 , return:  100.0
1648555047.8916006
(88, 80, 1)
Episode:  281 , return:  290.0
1648554909.1560113
(88, 80, 1)
Episode:  282 , return:  730.0
1648554537.5431588
(88, 80, 1)
Episode:  283 , return:  240.0
1648554916.5438313
(88, 80, 1)
Episode:  2

Episode:  401 , return:  190.0
1648557732.0448363
(88, 80, 1)
Episode:  402 , return:  360.0
1648557531.028401
(88, 80, 1)
Episode:  403 , return:  200.0
1648557845.2952785
(88, 80, 1)
Episode:  404 , return:  190.0
1648557917.022622
(88, 80, 1)
Episode:  405 , return:  280.0
1648557721.8823228
(88, 80, 1)
Episode:  406 , return:  200.0
1648557868.8404434
(88, 80, 1)
Episode:  407 , return:  660.0
1648557826.8564148
(88, 80, 1)
Episode:  408 , return:  120.0
1648558087.238657
(88, 80, 1)
Episode:  409 , return:  370.0
1648557742.4846509
(88, 80, 1)
Episode:  410 , return:  210.0
1648558030.1768534
(88, 80, 1)
Episode:  411 , return:  640.0
1648557419.8252583
(88, 80, 1)
Episode:  412 , return:  200.0
1648558103.7694392
(88, 80, 1)
Episode:  413 , return:  280.0
1648558004.5887074
(88, 80, 1)
Episode:  414 , return:  550.0
1648557852.4699073
(88, 80, 1)
Episode:  415 , return:  230.0
1648558080.8781831
(88, 80, 1)
Episode:  416 , return:  220.0
1648558132.1247866
(88, 80, 1)
Episode:  4

epsiloe_greedy() 함수에서 임의로 action을 선택할 확률 $\epsilon$ 을 episode가 증가할 수록 감소시키고 있기 때문에 action을 greedy하게 선택할 확률이 높아지게 된다. 그러므로 epsisode가 증가하면서 DQN의 main_network의 학습이 잘 되고있다면 return이 증가해야할 것이다. 

그러나 아래 288 episode에서 1060점을 기록하고 다음 episode에서는 150점을 기록한다. 이는 DQN이 아직 수렴하지 않았다는 것을 말하며 이 특정 state에서 update된 $Q_\theta (s,a)$의 모수 $\theta$가 다른 state에서는 오히려 낮은 $Q_\theta$값을 산출하는 원인이 되기 때문이다. DQN 모형은 atari 게임에서 학습이 잘 안되는 것으로 알려져있어, 다음 절에서 소개하는 DQN의 확장모형을 주로 사용한다.

약 4시간 소요 